# Browser Automation Homework ROUND 2
Due 7-16<br>
Completed by: **TK YOUR NAME**

We're going to the United States trademark data base and going to collect all of Nike (or any company's) active trademarks.

We only want the live trademarks, and we want the `serial` number and the link to an image of each trademark. Feel free to collect the `wordmark` if you like.


Note: if you get asked if you're a bot, just complete the challenges manually.

In [29]:
import os
import random
import time

from playwright.async_api import async_playwright, expect, Keyboard

In [2]:
os.makedirs('data/', exist_ok=True)

In [45]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [4]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.TK

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.TK

    return browser, page

In [70]:
driver, page = await open_browser()

In [71]:
url = 'https://tmsearch.uspto.gov/search/'
await page.TK(url)

<Response url='https://tmsearch.uspto.gov/search/' request=<Request url='https://tmsearch.uspto.gov/search/' method='GET'>>

Only search by the "Owner" of each trademark

In [73]:
xpath_search_options = '//TK'
filter = page.locator(xpath_search_options)

In [74]:
await filter.click()

In [75]:
xpath_option = '//TK'

In [76]:
await page.locator(xpath_option).click()

Find the search bar and input the search

In [77]:
xpath_searchbar = '//TK'

In [78]:
search = page.locator(xpath_searchbar)

In [79]:
# Change this if you like
company = 'Nike'

In [80]:
await search.TK(company)

Make the search

In [81]:
await page.TK

Filter to only live contracts
<br>In other words, filter out the dead ones.

In [82]:
xpath_filter = '//TK'

In [83]:
dead_button = page.locator(xpath_filter)

In [84]:
# click it
await dead_button.TK

# Save results

In [48]:
# how to save what the emulator sees
source = await page.content()
with open(f'data/trademarks_{company}.html', 'w') as f:
    f.write(source)

GO TO THE NEXT PAGE AND REPEAT

In [86]:
# this is the next page button
xpath_next = '//TK'

In [90]:
# What is the xpath for a result, make sure this is visible before going to next page.
xpath_result = '//TK'

In [66]:
next_button = page.locator(xpath_next)

In [67]:
await expect(next_button).to_be_visible()

In [68]:
await next_button.click()

In [95]:
# Here we'll put it all together and iterate through.
# There are many ways to do this
i = 1
collect = True
while collect:
    # check the next button is visible
    # if await page.is_visible(xpath_next, timeout=100):

    # note changes
    try:
        await expect(page.locator(xpath_next)).to_be_visible(timeout=1000)
        # click the next button
        next_button = page.locator(xpath_next)
        await next_button.click()

        # make sure the search result is visible
        await page.locator(xpath_result).is_visible()

        # save the contents
        source = await page.content()
        with open(f'data/trademarks_{company}_{i}.html', 'w') as f:
            f.write(source)
        i += 1
    # else:
    except Exception as e:
        collect = False
        break

## Parse the contents

In [107]:
import glob
import pandas as pd
from lxml import etree, html

In [101]:
# list the scraped pages to pars them
files = glob.glob('data/trademarks*.html')

In [140]:
# A trademark entry
xpath_trademark_cards = './/TK'
# the serial number
xpath_serial = './/TK'

In [177]:
data = []
for fn in files:
    # read into a XML tree from a string
    tree = html.fromstring(open(fn).read())
    trademarks = tree.findall(xpath_trademark_cards)
    for trademark in trademarks:
        serial = trademark.find(xpath_serial)
        # get the images also
        # img = trademark.find('.//img')
        # if img:
        #     img_url = img.get('src',)
        
        # note this change here:
        img_url = trademark.findall('.//img')
        if img_url:
            img_url = img_url[0].get('src')
        else:
            img_url = None
        
        row = {'serial' : serial.text, 'img_url': img_url}
        data.append(row)

/var/folders/sj/0bmsn0t57yb1tzg0xyg_9thm0000gq/T/ipykernel_47558/2386750351.py:10: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if img:


In [175]:
df = pd.DataFrame(data)
df.head(5)

,serial,img_url
0,88872671,https://tmcms-docs.uspto.gov/cases/75000070/ma...
1,98005404,https://tmcms-docs.uspto.gov/cases/75000070/ma...
2,88831783,https://tmcms-docs.uspto.gov/cases/75000070/ma...
3,90731895,https://tmcms-docs.uspto.gov/cases/75000070/ma...
4,90492296,https://tmcms-docs.uspto.gov/cases/75000070/ma...


In [176]:
df.to_csv('data/trademarks.csv', index=False)